# Création des données

La procédure que nous allons suivre est très simple :
- Nous allons tout d'abord prendre des photos qui ne respectent pas les conditions d'une pose correcte. Ces conditions seront appelées "NON_CORRECT" ("NON_CORRECT" est le **label** correspondant à cette condition). Nous prendrons des photographies à l'aide de la caméra.

- Ensuite, nous prendrons des photos qui respectent la condition d'une pose correcte. Le label correspondant sera nommé "CORRECT". Comme précédemment, des photographies seront prises et enregistrées sous ce label.

L'ensemble de ces résultats nous permettra de créer un ensemble de données appelé **dataset**. Quand nous aurons beaucoup d'images représentatives de ces deux conditions avec les labels associés, nous enverrons le dataset obtenu dans le GPU du jetson pour **entrainer** le réseau de neurones. Ce réseau de neurones nous permettra ensuite de prédire, en fonction des images issues de la caméra, si la personne filmée se trouve dans une condition d'une pose correcte ou non. Nous utiliserons ensuite le réseau entrainé pour gérer le score attribué à la bonne réalisation de la pose par la personne.

Tout cela sera fait pour différente position dans différents sports...

### Affichage de la caméra

Pour commencer, initialisons la caméra afin de créer un écran de visualisation.

**Pour réinitialiser la caméra :** sudo systemctl restart nvargus-daemon.service 

In [1]:
!sudo /home/jetson/notebook/ResetCam.sh

**Affiche les caméras disponnibles :**

In [2]:
!v4l2-ctl --list-devices

vi-output, imx219 7-0010 (platform:54080000.vi:0):
	/dev/video0

USB 2.0 Camera (usb-70090000.xusb-2.1):
	/dev/video1



In [3]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras

In [4]:
import tensorrt as trt
import numpy as np
import pycuda.driver as cuda
import threading
import ctypes
import time
import traitlets
import atexit
import cv2

class TRTInference(threading.Thread):
    def __init__(self,repertoire_engine, widget_image,type_camera,capture_device,capture_width,capture_height,display_width,display_height,fps,flip=0):
        threading.Thread.__init__(self)
        self.widget_image = widget_image
        self.type_camera = type_camera
        self.capture_device = capture_device
        self.capture_width = capture_width
        self.capture_height = capture_height
        self.display_width = display_width
        self.display_height = display_height
        self.fps = fps
        self.flip = flip
        self.camera_on = False

        # Initialisation des variables de la caméra
        self._running = False
        self.image_pour_widget = np.zeros((capture_width, capture_height, 3), dtype=np.int32)
        self.image_origine = np.zeros((capture_width, capture_height, 3), dtype=np.int32)
        self.image = np.zeros((256, 256, 3), dtype=np.int32)
        
        if self.type_camera.find("CSI")>=0:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_CSI(),cv2.CAP_GSTREAMER)
        else:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_USB(),cv2.CAP_GSTREAMER)
        if self.cap.isOpened():
            print("Caméra initialisée")
        else:
            print("Erreur d'ouverture du flux vidéo")
        atexit.register(self.cap.release)

        # Initialisation du runtime TensorRT
        #self.logger = MyLogger()
        self.logger = trt.Logger(trt.Logger.INFO)
        trt.init_libnvinfer_plugins(self.logger, namespace="")
        self.runtime = trt.Runtime(self.logger)
        
        # Chargement du moteur
        print("Chargement du moteur...")
        with open(repertoire_engine, "rb") as f:
            self.engine = self.runtime.deserialize_cuda_engine(f.read())
        
        self.context = self.engine.create_execution_context()
 
        #Initialisation du context Cuda et du contexte TensorRT 
        cuda.init()
        self.cudactx = cuda.Device(0).retain_primary_context()
        self.cudactx.push()
        self.context.debug_sync = True
        
        # Réservation de la mémoire pour l'entrée
        print("Allocation mémoire...")
        size_input = trt.volume(self.engine.get_binding_shape(0))*self.engine.max_batch_size
        self.input_host_mem = cuda.pagelocked_empty(size_input, np.int32)
        self.input_device_mem = cuda.mem_alloc(self.input_host_mem.nbytes)
       
        # Réservation de la mémoire pour les sorties
        self.output_device_mem = [];
        format_sorties = [];
        types_sorties = [];

        for i in range(self.engine.num_bindings):
            if not self.engine.binding_is_input(i):
                size_output = trt.volume(self.engine.get_binding_shape(i))*self.engine.max_batch_size
                output_hm = cuda.pagelocked_empty(size_output,trt.nptype(self.engine.get_binding_dtype(i)))
                self.output_device_mem.append(cuda.mem_alloc(output_hm.nbytes))
                format_sorties.append(self.engine.get_binding_shape(i))
                types_sorties.append(trt.nptype(self.engine.get_binding_dtype(i)))

        # Récupère les adresses en GPU des buffers entrées / sorties
        binding_entree = int(self.input_device_mem)
        binding_sorties = []

        for output_ in self.output_device_mem:
            binding_sorties.append(int(output_))
        self.bindings = [binding_entree, binding_sorties[0]]

        # Allocation de la mémoire hote pour les sorties
        self.output_host_mem = []
        for i in range(len(self.output_device_mem)):
            self.output_host_mem.append(np.zeros(format_sorties[i],types_sorties[i]))
        
        # Input tensor
        self.image = np.zeros((256, 256, 3), dtype=trt.nptype(self.engine.get_binding_dtype(0)))
        self.image = self.image.astype(np.int32)       
        self.cudactx.pop()

    # Inférence
    def Calcul(self):
        # Copie de l'image dans le tenseur d'entrée
        x = self.image.astype(np.int32)
        x = np.expand_dims(x,axis=0)                    # (1,256,256,3)
        np.copyto(self.input_host_mem,x.ravel())
        
        # Transfert de l'entrée vers le GPU
        self.cudactx = cuda.Device(0).retain_primary_context()
        self.cudactx.push()
        cuda.memcpy_htod(self.input_device_mem, self.input_host_mem)
        
        # Appel du modèle
        self.context.execute_v2(bindings=self.bindings)
        
        # Récupération des sorties
        for i in range(len(self.output_host_mem)):
            cuda.memcpy_dtoh(self.output_host_mem[i], self.output_device_mem[i])
        self.cudactx.pop()

    # Lecture d'une frame
    def capture_image(self):
        re, image = self.cap.read()
        if re:
            return image
        else:
            return self.image_pour_widget
        
    def run(self):
        while True:
            if self.camera_on is True:
                self.image_pour_widget = self.capture_image()
                self.image_origine = bgr8_to_jpeg(self.image_pour_widget)
                self.image = cv2.resize(self.image_pour_widget,(int(256),int(256)))              
                self.Calcul()
                
                # Récupère les coordonnees XY
                coordonneesXY = self.output_host_mem[0][0][0][:][:]
                
                # Place les points
                width, height = (self.display_width,self.display_height)

                nez = [int(coordonneesXY[0][1]*width),int(coordonneesXY[0][0]*height)]
                oeuil_gauche = [int(coordonneesXY[1][1]*width),int(coordonneesXY[1][0]*height)]
                oeuil_droit = [int(coordonneesXY[2][1]*width),int(coordonneesXY[2][0]*height)]
                oreille_gauche = [int(coordonneesXY[3][1]*width),int(coordonneesXY[3][0]*height)]
                oreille_droite = [int(coordonneesXY[4][1]*width),int(coordonneesXY[4][0]*height)]
                epaule_gauche = [int(coordonneesXY[5][1]*width),int(coordonneesXY[5][0]*height)]
                epaule_droite = [int(coordonneesXY[6][1]*width),int(coordonneesXY[6][0]*height)]
                coude_gauche = [int(coordonneesXY[7][1]*width),int(coordonneesXY[7][0]*height)]
                coude_droite = [int(coordonneesXY[8][1]*width),int(coordonneesXY[8][0]*height)]
                poignet_gauche = [int(coordonneesXY[9][1]*width),int(coordonneesXY[9][0]*height)]
                poignet_droite = [int(coordonneesXY[10][1]*width),int(coordonneesXY[10][0]*height)]
                hanche_gauche = [int(coordonneesXY[11][1]*width),int(coordonneesXY[11][0]*height)]
                hanche_droite = [int(coordonneesXY[12][1]*width),int(coordonneesXY[12][0]*height)]
                genou_gauche = [int(coordonneesXY[13][1]*width),int(coordonneesXY[13][0]*height)]
                genou_droite = [int(coordonneesXY[14][1]*width),int(coordonneesXY[14][0]*height)]
                cheville_gauche = [int(coordonneesXY[15][1]*width),int(coordonneesXY[15][0]*height)]
                cheville_droite = [int(coordonneesXY[16][1]*width),int(coordonneesXY[16][0]*height)]


                centre_yeux = [((int(0.5*(coordonneesXY[1][1]*width+int(coordonneesXY[2][1]*width))))),((int(0.5*(coordonneesXY[1][0]*height+int(coordonneesXY[2][0]*height)))))]
                centre_epaules = [((int(0.5*(coordonneesXY[5][1]*width+int(coordonneesXY[6][1]*width))))),((int(0.5*(coordonneesXY[5][0]*height+int(coordonneesXY[6][0]*height)))))]
                centre_hanches =  [((int(0.5*(coordonneesXY[11][1]*width+int(coordonneesXY[12][1]*width))))),((int(0.5*(coordonneesXY[11][0]*height+int(coordonneesXY[12][0]*height)))))]


                # Lignes nez-centre des yeux
                self.image_pour_widget = cv2.line(self.image_pour_widget, (nez[0],nez[1]),(centre_yeux[0],centre_yeux[1]),color=(0, 0, 255), thickness=2)

                # Lignes centre des yeux - oueil gauche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (oeuil_gauche[0],oeuil_gauche[1]),(centre_yeux[0],centre_yeux[1]),color=(0, 0, 255), thickness=2)

                # Lignes centre des yeux - oueil droit
                self.image_pour_widget = cv2.line(self.image_pour_widget, (oeuil_droit[0],oeuil_droit[1]),(centre_yeux[0],centre_yeux[1]),color=(0, 0, 255), thickness=2)

                # Lignes oeuil_gauche - oreille gauche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (oeuil_gauche[0],oeuil_gauche[1]),(oreille_gauche[0],oreille_gauche[1]),color=(0, 0, 255), thickness=2)

                # Lignes oeuil_droit - oreille droite
                self.image_pour_widget = cv2.line(self.image_pour_widget, (oeuil_droit[0],oeuil_droit[1]),(oreille_droite[0],oreille_droite[1]),color=(0, 0, 255), thickness=2)

                # Lignes nez-centre_epaules
                self.image_pour_widget = cv2.line(self.image_pour_widget, (nez[0],nez[1]),(centre_epaules[0],centre_epaules[1]),color=(0, 0, 255), thickness=2)

                # Ligne centre_epaules - epaule_droit
                self.image_pour_widget = cv2.line(self.image_pour_widget, (epaule_droite[0],epaule_droite[1]),(centre_epaules[0],centre_epaules[1]),color=(0, 0, 255), thickness=2)

                # Ligne centre_epaules - epaule_gauche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (epaule_gauche[0],epaule_gauche[1]),(centre_epaules[0],centre_epaules[1]),color=(0, 0, 255), thickness=2)

                # Ligne épaule_gauche - coude_gouche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (epaule_gauche[0],epaule_gauche[1]),(coude_gauche[0],coude_gauche[1]),color=(0, 0, 255), thickness=2)

                # Ligne coude_gouche - poignet_gauche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (poignet_gauche[0],poignet_gauche[1]),(coude_gauche[0],coude_gauche[1]),color=(0, 0, 255), thickness=2)

                # Ligne épaule_droite - coude_doit
                self.image_pour_widget = cv2.line(self.image_pour_widget, (epaule_droite[0],epaule_droite[1]),(coude_droite[0],coude_droite[1]),color=(0, 0, 255), thickness=2)

                # Ligne coude_droite - poignet_droite
                self.image_pour_widget = cv2.line(self.image_pour_widget, (poignet_droite[0],poignet_droite[1]),(coude_droite[0],coude_droite[1]),color=(0, 0, 255), thickness=2)

                # Ligne centre_epaules - centre_hanche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (centre_hanches[0],centre_hanches[1]),(centre_epaules[0],centre_epaules[1]),color=(0, 0, 255), thickness=2)

                # Ligne centre_hanche - hanche_gauche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (centre_hanches[0],centre_hanches[1]),(hanche_gauche[0],hanche_gauche[1]),color=(0, 0, 255), thickness=2)

                # Ligne centre_hanche - hanche_gauche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (centre_hanches[0],centre_hanches[1]),(hanche_droite[0],hanche_droite[1]),color=(0, 0, 255), thickness=2)

                # Ligne hanche_gauche - genou_gauche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (genou_gauche[0],genou_gauche[1]),(hanche_gauche[0],hanche_gauche[1]),color=(0, 0, 255), thickness=2)

                # Ligne hanche_droite - genou_droite
                self.image_pour_widget = cv2.line(self.image_pour_widget, (genou_droite[0],genou_droite[1]),(hanche_droite[0],hanche_droite[1]),color=(0, 0, 255), thickness=2)

                # Ligne genou_droite - cheville_droite
                self.image_pour_widget = cv2.line(self.image_pour_widget, (genou_droite[0],genou_droite[1]),(cheville_droite[0],cheville_droite[1]),color=(0, 0, 255), thickness=2)

                # Ligne genou_gauche - cheville_gauche
                self.image_pour_widget = cv2.line(self.image_pour_widget, (genou_gauche[0],genou_gauche[1]),(cheville_gauche[0],cheville_gauche[1]),color=(0, 0, 255), thickness=2)                
                
                self.widget_image.value = bgr8_to_jpeg(self.image_pour_widget)
                time.sleep(0.001)

    # Définition du pipeline pour la caméra CSI
    def _gstreamer_pipeline_CSI(self):
        return("nvarguscamerasrc sensor-id=%d ! "
                "video/x-raw(memory:NVMM),"
                "width=(int)%d,height=(int)%d,"
                "format=(string)NV12, framerate=(fraction)%d/1 ! "
                "nvvidconv flip-method=%d ! "
                "video/x-raw,"
                "width=(int)%d,height=(int)%d,"
                "format=(string)BGRx ! videoconvert ! "
                "video/x-raw, format=(string)BGR ! "
                "appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip, self.display_width,self.display_height))

    # Définition du pipeline pour la USB
    def _gstreamer_pipeline_USB(self):
        return("v4l2src device=/dev/video%d ! "
               "video/x-raw, width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
               "videoflip method=%d ! "
               "videoconvert ! "
               "video/x-raw, format=(string)BGR ! appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip))            

    # Routine pour arrêter le Thread
    def raise_exception(self):
        for id, thread in threading._active.items():
            if thread is self:
                thread_id = id
        res = ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id,ctypes.py_object(SystemExit))
        if res > 1:
            ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id, 0)
            print('Exception raise failure')

    def destroy(self):
        self.cudactx.detach()

### Répertoires de sauvegarde

Maintenant, nous allons créer un répertoire qui nous permettra de sauvegarder nos données.

Nous allons nommer ce répertoire ``dataset``. Il va contenir deux sous-répertoires : ``CORRECT`` et ``NON_CORRECT``. Les images correspondantes à chaque condition seront sauvegardées dans ces sous-répertoires.

Ces deux répertoires sont contenus dans un répertoire principal nommé en fonction du nom de la position qui est entrainée.

In [6]:
nom_position = "Corbeau"

In [7]:
import os

repertoire_correct = 'dataset/' + nom_position + '/CORRECT'
repertoire_non_correct = 'dataset/' + nom_position + '/NON_CORRECT'

# On utilise lex exeptions car si les réperoires exsitent déjà cela provoquera une erreur
try:
    os.makedirs(repertoire_correct)
    os.makedirs(repertoire_non_correct)
except FileExistsError:
    print('Réperoires non créés car ils existent déjà')

Vous devriez maintenant voir le réperoire dataset ainsi que les répertoires associés.

### Interface d'acquisition

Nous allons maintenant afficher quelques boutons afin d'avoir un interface pour prendre les photographies et décider à quelle **classe** (c'est à dire quel label) appartient chaque photo. Nous allons également indiquer combien d'images sont contenues dans chaque classe.

In [8]:
# Librairies IPython pour les widgets
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

layout_bouton = widgets.Layout(width='200px', height='64px')
bouton_correct = widgets.Button(description='Ajouter correct', button_style='success', layout=layout_bouton)
bouton_non_correct = widgets.Button(description='Ajouter non_correct', button_style='danger', layout=layout_bouton)
nombre_correct = widgets.IntText(layout=layout_bouton, value=len(os.listdir(repertoire_correct)))
nombre_non_correct = widgets.IntText(layout=layout_bouton, value=len(os.listdir(repertoire_non_correct)))

display(widgets.HBox([nombre_correct, bouton_correct]))
display(widgets.HBox([nombre_non_correct, bouton_non_correct]))

Pour le moment ces boutons n'ont aucune action associée. Il faut que nous attachions des fonctions permettant de sauvegarder une image à chaque fois qu'un clic de souris est détecté sur le bouton. Pour cela nous allons utiliser l'évènement ``on_click``. Nous sauvegarderons alors la valeur du widget ``Image`` (plutôt que celui du widget camera car l'image est déjà compressée au format JPEG !)

Pour être sûr de ne pas répéter le nom des fichiers, nous allons utiliser le package ``uuid``.

In [9]:
from uuid import uuid1

# Fonction permettant de sauvegarder une image avec un nom de fichier unique
def sauvegarde_image(repertoire):
    chemin_image = os.path.join(repertoire, str(uuid1()) + '.jpg')
    with open(chemin_image, 'wb') as f:
        f.write(trt_inference_wrapper.image_origine)

# Fonction appellée lors de l'appui sur le bouton "Ajouter correct"
def sauvegarde_correct():
    global repertoire_correct, nombre_correct
    sauvegarde_image(repertoire_correct)
    nombre_correct.value = len(os.listdir(repertoire_correct))
    
# Fonction appellée lors de l'appui sur le bouton "Ajouter bloquer"
def sauvegarde_non_correct():
    global repertoire_non_correct, nombre_non_correct
    sauvegarde_image(repertoire_non_correct)
    nombre_non_correct.value = len(os.listdir(repertoire_non_correct))
    
# Création des liens entre les fonctions et les évènements "on_click" des boutons
bouton_correct.on_click(lambda x: sauvegarde_correct())
bouton_non_correct.on_click(lambda x: sauvegarde_non_correct())

Les butons peuvent maintenant sauvegarder nos images dans les bons répertoires ! 

Il est temps de récupérer des données :

1. Placer la personne dans les conditions "non correct" et cliquer sur le bouton ``Ajouter non correct``
2. Placer la personne dans les conditions "correct" et cliquer sur le bouton ``Ajouter correct``
3. Répéter les étapes 1 et 2

Quelques astuces :

1. Utiliser différentes orientations
2. Utiliser différents éclairages

Dans l'idéal, plus le dataset est important et plus l'IA sera capable de détecter les positions dans la vie réelle. Il est important d'avoir des données *variées* et non pas uniquement beaucoup de données. Vous aurez probablement besoin d'au moins 100 images par classe.

In [10]:
from jetbot import bgr8_to_jpeg

# Création de l'interface
widget_but1 = ipywidgets.VBox([nombre_correct, bouton_correct])
widget_but2 = ipywidgets.VBox([nombre_non_correct, bouton_non_correct])
widget_but = ipywidgets.VBox([widget_but1,widget_but2])
image_widget = widgets.Image(format='jpeg', width=640, height=480)

widget_global = ipywidgets.HBox([image_widget, widget_but])


trt_inference_wrapper = TRTInference(repertoire_engine="model_jetson.engine",
                        widget_image=image_widget,
                        type_camera="USB",capture_device=1,
                        capture_width=1280,capture_height=960,
                        display_width=1280,display_height=960,
                        fps=30,flip=0)

trt_inference_wrapper.start()
trt_inference_wrapper.camera_on = True

# Affichage de l'interface
display(widget_global)

Caméra initialisée
Chargement du moteur...
Allocation mémoire...


In [11]:
!zip -r dataset.zip dataset

  adding: dataset/ (stored 0%)
  adding: dataset/Corbeau/ (stored 0%)
  adding: dataset/Corbeau/NON_CORRECT/ (stored 0%)
  adding: dataset/Corbeau/NON_CORRECT/dfc62fa2-eb60-11ee-9231-48b02d51ecb9.jpg (deflated 0%)
  adding: dataset/Corbeau/NON_CORRECT/b3b7f67a-eb60-11ee-9231-48b02d51ecb9.jpg (deflated 0%)
  adding: dataset/Corbeau/NON_CORRECT/cbf90260-eb60-11ee-9231-48b02d51ecb9.jpg (deflated 0%)
  adding: dataset/Corbeau/NON_CORRECT/d778ec68-eb60-11ee-9231-48b02d51ecb9.jpg (deflated 0%)
  adding: dataset/Corbeau/NON_CORRECT/aaf7332a-eb60-11ee-9231-48b02d51ecb9.jpg (deflated 0%)
  adding: dataset/Corbeau/NON_CORRECT/cf54075c-eb60-11ee-9231-48b02d51ecb9.jpg (deflated 0%)
  adding: dataset/Corbeau/NON_CORRECT/e104de04-eb60-11ee-9231-48b02d51ecb9.jpg (deflated 0%)
  adding: dataset/Corbeau/NON_CORRECT/b251f0b0-eb60-11ee-9231-48b02d51ecb9.jpg (deflated 0%)
  adding: dataset/Corbeau/NON_CORRECT/df30036a-eb60-11ee-9231-48b02d51ecb9.jpg (deflated 0%)
  adding: dataset/Corbeau/NON_CORRECT/921f